In [1]:
import syft as sy
import numpy as np
from syft.service.policy.policy import OutputPolicyExecuteOnce, BeachPolicy

In [2]:
node = sy.orchestra.launch(name="test-domain-1", dev_mode=True, reset=True)

Staging Protocol Changes...
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=870af94b12e94873a83750f555ef595d in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
root_client = node.login(email="info@openmined.org", password="changethis")

Logged into <test-domain-1: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
root_client.register(
    name="Jane Doe",
    email="jane@caltech.edu",
    password="abc123",
    password_verify="abc123",
    institution="Caltech",
    website="https://www.caltech.edu/",
)

SyftSuccess: User 'Jane Doe' successfully registered! To see users, run `[your_client].users`

In [5]:
domain_client = node.login(email="jane@caltech.edu", password="abc123")

Logged into <test-domain-1: High side Domain> as <jane@caltech.edu>


In [6]:
dataset = sy.Dataset(
    name="test",
    asset_list=[
        sy.Asset(
            name="test",
            data=np.array([1, 2, 3]),
            mock=np.array([1, 1, 1]),
            mock_is_real=False,
        )
    ],
)
root_client.upload_dataset(dataset)


  0%|                                                                                                                 | 0/1 [00:00<?, ?it/s]

Uploading: test


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.85it/s]


SyftSuccess: Dataset uploaded to 'test-domain-1'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
asset = domain_client.datasets[0].assets[0]

In [8]:
asset

syft.service.dataset.dataset.Asset

In [9]:
# str == str

In [10]:
policy = BeachPolicy(static_arg=asset, prompt=str)

In [11]:
from syft import serialize, deserialize

In [12]:
# ser = serialize(policy)

# pol = deserialize(ser)

In [13]:
@sy.syft_function(
    input_policy=policy,
    output_policy=OutputPolicyExecuteOnce()
)
def my_vertex_func(
    static_arg,
    prompt:str
):  
    return prompt + "DEF"
    

SyftSuccess: Syft function 'my_vertex_func' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [14]:
request = domain_client.code.request_code_execution(my_vertex_func)

In [15]:
request

syft.service.request.request.Request

In [16]:
root_client.requests[-1].approve()

Approving request for domain test-domain-1


SyftSuccess: Request 3ba54a613ed747d8b5086b70f6495071 changes applied

In [20]:
res = domain_client.code.my_vertex_func(static_arg=asset, prompt=1)

TypeCheckError: int is not an instance of str

In [18]:
res

```python
TwinPointer(Real)
```
'DEFDEF'

In [19]:
res.get()

'DEFDEF'

In [14]:
# def my_vertex_func(
#     ,
#     prompt: str,
#     max_tokens: int,
#     temperature: float,
#     top_p: float,
#     top_k: int,
#     raw_response: bool
# ):
